In [1]:
import psycopg2 as pg
import pandas.io.sql as psql



In [2]:
connection = pg.connect(
    host="localhost",
    dbname="app",
    user="football", 
    password="football")

In [22]:
split_dates = ['2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01']
end_dates = ['2022-11-30', '2022-12-31', '2023-01-31', '2023-02-28', '2023-03-31']

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb

train_features = [
    "goals_last_3_home_matches",
    "goals_last_3_away_matches",
    "more_than_6_goals_last_3_home_matches",
    "num_matches_greater_2_goals_last_3_home_matches",
    "number_matches_atleast_one_team_not_scored_last_3_home_matches",
    "more_than_6_goals_last_3_away_matches",
    "num_matches_greater_2_goals_last_3_away_matches",
    "num_goals_last_match",
    "num_matches_greater_0_away_goals_last_3_away_matches",
    "home_team_points",
    "away_team_points"    
]


for i in range(len(split_dates)):
    split_date = split_dates[i]
    end_date = end_dates[i]
    train = psql.read_sql(f"SELECT * FROM data_for_statistics_preparation_total where fixture_timestamp_utc < '{split_date}'", connection)
    test = psql.read_sql(f"SELECT f.*, o.odd, o.betting_type FROM data_for_statistics_preparation_total f left join public.module1_odd o on o.fixture_id = f.fixture_id and betting_type = 'under25' and bookmaker = 'Bet365' where f.fixture_timestamp_utc between '{split_date}' and '{end_date}';", connection)
    train["under25"] = [True if res < 3 else False for res in train['total']]
    actuals = [True if res < 3 else False for res in test['total']]

    categorical_features=[]

    preprocessor = ColumnTransformer(
        transformers=[
            ("categorical", OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ]
    )
    param_dict = {
        "max_delta_step": 1,
        "scale_pos_weight": 1
    }

    classifier = xgb.XGBClassifier(
        max_delta_step=param_dict["max_delta_step"],
        scale_pos_weight=param_dict["scale_pos_weight"],
        objective="binary:logistic"
    )

    pipeline = Pipeline(
        steps=[
            #('preprocessor', preprocessor),
            ('classifier', classifier)
        ]
    )

    y = train[["under25"]].values.ravel()
    x = train[train_features]

    pipeline.fit(x, y)

    from sklearn.metrics import confusion_matrix

    preds = pipeline.predict(test[train_features])
    predicted_probs  = pipeline.predict_proba(test[train_features])[:, 1]

    tn, fp, fn, tp = confusion_matrix(actuals, preds).ravel()
    print("Eval Start: ", split_date)
    print("Eval End: ", end_date)
    print("Accuracy :", (tp + tn) / (tn + fp + fn + tp))
    print("Recall :", tp/ (tp + fn))

    test["own_odd"] = 1/predicted_probs
    filtered_df = test[test["odd"] > 0]
    thres = 1.0
    filtered_df = filtered_df[filtered_df["odd"] - filtered_df["own_odd"] > thres]

    money = 0
    for _, row in filtered_df.iterrows():
        if row.total <= 2:
            money += row.odd - 1
        else:
            money -= 1

    print("Number Games: ", filtered_df.shape[0])
    print("P/L", money)


Eval Start:  2022-11-01
Eval End:  2022-11-30
Accuracy : 0.537628659270151
Recall : 0.5370985603543743
Number Games:  101
P/L 0.09999999999999876
Eval Start:  2022-12-01
Eval End:  2022-12-31
Accuracy : 0.5506155950752394
Recall : 0.6964757709251101
Number Games:  69
P/L -3.299999999999999
Eval Start:  2023-01-01
Eval End:  2023-01-31
Accuracy : 0.5351652574942352
Recall : 0.6470370370370371
Number Games:  60
P/L 40.50000000000001
Eval Start:  2023-02-01
Eval End:  2023-02-28
Accuracy : 0.5280412714261941
Recall : 0.8674850487881649
Number Games:  82
P/L -9.500000000000002
Eval Start:  2023-03-01
Eval End:  2023-03-31
Accuracy : 0.5326411605745982
Recall : 0.7320404261130838
Number Games:  86
P/L 22.65


In [4]:
train["under25"] = [True if res < 3 else False for res in train['total']]
actuals = [True if res < 3 else False for res in test['total']]

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb

train_features = [
    "goals_last_3_home_matches",
    "goals_last_3_away_matches",
    "more_than_6_goals_last_3_home_matches",
    "num_matches_greater_2_goals_last_3_home_matches",
    "number_matches_atleast_one_team_not_scored_last_3_home_matches",
    "more_than_6_goals_last_3_away_matches",
    "num_matches_greater_2_goals_last_3_away_matches",
    "num_goals_last_match",
    "num_matches_greater_0_away_goals_last_3_away_matches",
    "home_team_points",
    "away_team_points"    
]

categorical_features=[]

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)
param_dict = {
    "max_delta_step": 1,
    "scale_pos_weight": 1
}

classifier = xgb.XGBClassifier(
    max_delta_step=param_dict["max_delta_step"],
    scale_pos_weight=param_dict["scale_pos_weight"],
    objective="binary:logistic"
)

pipeline = Pipeline(
    steps=[
        #('preprocessor', preprocessor),
        ('classifier', classifier)
    ]
)

y = train[["under25"]].values.ravel()
x = train[train_features]

pipeline.fit(x, y)

Pipeline(steps=[('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=1,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [14]:
from sklearn.metrics import confusion_matrix

preds = pipeline.predict(test[train_features])
predicted_probs  = pipeline.predict_proba(test[train_features])[:, 1]

tn, fp, fn, tp = confusion_matrix(actuals, preds).ravel()
print("Accuracy :", (tp + tn) / (tn + fp + fn + tp))
print("Recall :", tp/ (tp + fn))

Accuracy : 0.5361771487636161
Recall : 0.7166511194029851


In [13]:
test["own_odd"] = 1/predicted_probs
filtered_df = test[test["odd"] > 0]
thres = 1.0
filtered_df = filtered_df[filtered_df["odd"] - filtered_df["own_odd"] > thres]

money = 0
for _, row in filtered_df.iterrows():
    if row.total <= 2:
        money += row.odd - 1
    else:
        money -= 1

print("Number Games: ", filtered_df.shape[0])
money

Number Games:  360


26.510000000000034

# System


In [15]:
test = psql.read_sql(f"""SELECT fixture.fixture_id, fixture.fixture_date_utc,to_char(fixture.fixture_timestamp_utc, 'HH24:MI:SS'::text) AS fixture_time_utc, league.country,
 league.league_name, 
    league.round,
    fixture.home_team_name,
    fixture.away_team_name,
    fixture.goals_away + fixture.goals_home as total,
    odds.odd
   FROM module1_under25 calc
     JOIN module1_fixture fixture ON fixture.fixture_id = calc.fixture_id
     LEFT JOIN module1_leagueseason league ON fixture.league_season_id = league.league_season_id
     LEFT JOIN currentodds odds ON odds.fixture_id::text = calc.fixture_id AND odds.betting_type = 'under25'::text AND odds.bookmaker = 'Bet365'::text
  WHERE fixture.fixture_date_utc  between '{split_date}' and '2023-03-10'
  ORDER BY fixture.fixture_timestamp_utc;""", connection)

In [16]:
test

,fixture_id,fixture_date_utc,fixture_time_utc,country,league_name,round,home_team_name,away_team_name,total,odd
0,967617,2022-10-31,07:00:00,China,League One,Regular Season - 1,BIT,Shenyang Urban,3.0,2.50
1,967614,2022-10-31,07:00:00,China,League One,Regular Season - 1,Nantong Zhiyun,Nanjing City,0.0,2.05
2,967615,2022-10-31,07:00:00,China,League One,Regular Season - 1,Heilongjiang Lava Spring,Hebei Kungfu,2.0,NaN
3,967616,2022-10-31,07:00:00,China,League One,Regular Season - 1,Jiangxi Liansheng,Sichuan Jiuniu,1.0,2.10
4,959626,2022-10-31,10:45:00,Iran,Azadegan League,Regular Season - 1,Arman Gohar,Mes Shahr-e Babak,5.0,1.40
...,...,...,...,...,...,...,...,...,...,...
1178,881491,2023-03-06,14:30:00,Russia,First League,Regular Season - 1,Urozhay,Rubin,3.0,NaN
1179,879640,2023-03-06,17:00:00,Czech-Republic,FNL,Regular Season - 1,Sparta Praha II,Slavia Praha II,2.0,2.05
1180,875755,2023-03-07,19:45:00,England,League One,Regular Season - 1,Cheltenham,Lincoln,0.0,1.44
1181,874261,2023-03-08,12:00:00,Bosnia,Premijer Liga,Regular Season - 1,FK Sarajevo,Zeljeznicar Sarajevo,0.0,1.62


In [17]:
filtered_df = test[test["odd"] > 0]

money = 0
for _, row in filtered_df.iterrows():
    if row.total <= 2:
        money += row.odd - 1
    else:
        money -= 1

print("Number Games: ", filtered_df.shape[0])
money

Number Games:  986


-121.90999999999987

In [ ]:
import http.client
import json
from dot_env import load_dotenv

load_dotenv()

api_result = []

for fixture in test["fixture_id"]:
    conn = http.client.HTTPSConnection("v3.football.api-sports.io")

    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': os.getenv('FOOTBALL_API_KEY')
        }

    conn.request("GET", f"/predictions?fixture={fixture}", headers=headers)

    res = conn.getresponse()
    data = res.read()
    res = json.loads(data.decode("utf-8"))
    pred = res["response"][0]["predictions"]["under_over"]
    api_result.append(pred)
    print(f"Fixture: {fixture}, Prediction: {pred}")
#print(data.decode("utf-8")[0]["response"])

ModuleNotFoundError: No module named 'dot_env'

In [ ]:
[True if res in ["+2.5", "+3.5", "+4.5"] else False for res in api_result]

In [ ]:
product_category = psql.read_sql_query('SELECT * FROM xgstatistics limit 100', connection)
product_category.head()

/opt/conda/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,fixture_id,home_goals_for_sma500,home_goals_against_sma500
0,172691,NaN,NaN
1,172693,2.0,2.0
2,172692,1.0,1.5
3,172695,1.0,1.0
4,172696,0.8,1.3


In [ ]:
import mlflow